# TicTacTwo: Estimate Total Number of Possible Positions

This notebook computes a combinatorial estimate for the TicTacTwo variant in this repo.

Assumptions used:
- Board is 5x5 (25 cells).
- Each player can have at most 4 pieces on board.
- Legal turn progression implies only these count pairs are reachable during play:
  (0,0), (1,0), (1,1), (2,1), (2,2), (3,2), (3,3), (4,3), (4,4).
- Active 3x3 window can be in 9 positions.
- For (4,4), either side can be to move (because later turns are move/shift only).


In [11]:
from math import comb

BOARD_CELLS = 25
ACTIVE_WINDOW_POSITIONS = 9

legal_count_pairs = [
    (0, 0), (1, 0), (1, 1), (2, 1), (2, 2),
    (3, 2), (3, 3), (4, 3), (4, 4),
]

def arrangements_for_counts(x_count: int, o_count: int) -> int:
    return comb(BOARD_CELLS, x_count) * comb(BOARD_CELLS - x_count, o_count)

board_only_legal = sum(arrangements_for_counts(x, o) for x, o in legal_count_pairs)
positions_with_active = board_only_legal * ACTIVE_WINDOW_POSITIONS

# Side-to-move is uniquely determined for all count pairs except (4,4),
# where both players can be to move after placement phase.
extra_turn_states_at_4_4 = arrangements_for_counts(4, 4) * ACTIVE_WINDOW_POSITIONS
states_including_turn = positions_with_active + extra_turn_states_at_4_4

# Looser upper bound: all (x,o) with 0<=x<=4 and 0<=o<=4, ignoring turn legality.
board_only_unconstrained = sum(
    arrangements_for_counts(x, o)
    for x in range(5)
    for o in range(5)
)

results = {
    "board_only_legal_count_pairs": board_only_legal,
    "positions_with_active_window": positions_with_active,
    "game_states_including_side_to_move": states_including_turn,
    "board_only_unconstrained_upper_bound": board_only_unconstrained,
}
results


{'board_only_legal_count_pairs': 96691476,
 'positions_with_active_window': 870223284,
 'game_states_including_side_to_move': 1551615534,
 'board_only_unconstrained_upper_bound': 120030201}

## Interpretation

Key estimate to use for planning search/storage work:
- `positions_with_active_window` = 870,223,284
- `game_states_including_side_to_move` = 1,551,615,534

These are combinatorial counts under rules-consistent piece totals, not the exact number of states reachable from the initial state after enforcing all dynamic constraints.


## Reachable-Subset Bounds From Initial State

This section adds empirical bounds for *actually reachable* states from the initial position.

- Lower bound: exact unique states reached up to a fixed ply depth via forward expansion.
- Upper bound: the combinatorial total from the previous section.


In [12]:
import json
import subprocess

def run_node(js_code: str) -> str:
    result = subprocess.run(
        ["node", "-e", js_code],
        check=True,
        capture_output=True,
        text=True,
    )
    return result.stdout.strip()


In [13]:
max_depth = 8

depth_js = r"""
const { createInitialState, getStateKey, getAvailableActions, applyAction, getOpponent, getWinner, isDraw } = require("../../dist/game");

function depthCounts(maxDepth){
  let layer = [{state:createInitialState(), player:"X"}];
  const seen = new Set();
  const rows = [];

  for(let depth=0; depth<=maxDepth; depth++){
    const layerMap = new Map();
    for(const n of layer){
      const k = getStateKey(n.state,n.player);
      if(!layerMap.has(k)) layerMap.set(k,n);
    }
    layer = Array.from(layerMap.values());

    for(const n of layer) seen.add(getStateKey(n.state,n.player));

    let terminal = 0;
    const nextMap = new Map();
    if(depth < maxDepth){
      for(const n of layer){
        if(getWinner(n.state) || isDraw(n.state)) { terminal++; continue; }
        const np = getOpponent(n.player);
        for(const a of getAvailableActions(n.state,n.player)){
          const ns = applyAction(n.state,a,n.player);
          const nk = getStateKey(ns,np);
          if(!nextMap.has(nk)) nextMap.set(nk,{state:ns,player:np});
        }
      }
    }

    rows.push({
      depth,
      uniqueAtDepth: layer.length,
      cumulativeUniqueToDepth: seen.size,
      terminalAtDepth: terminal,
      uniqueNextDepth: nextMap.size
    });

    layer = Array.from(nextMap.values());
  }
  return rows;
}

console.log(JSON.stringify(depthCounts(8)));
"""

rows = json.loads(run_node(depth_js))
for r in rows:
    print(r)

lower_bound = rows[-1]["cumulativeUniqueToDepth"]
upper_bound = 1551615534
print()
print({
    "reachable_lower_bound_from_depth_limited_expansion": lower_bound,
    "reachable_upper_bound_combinatorial": upper_bound,
    "fraction_of_upper_bound_covered_by_depth_prefix": lower_bound / upper_bound,
})


{'depth': 0, 'uniqueAtDepth': 1, 'cumulativeUniqueToDepth': 1, 'terminalAtDepth': 0, 'uniqueNextDepth': 9}
{'depth': 1, 'uniqueAtDepth': 9, 'cumulativeUniqueToDepth': 10, 'terminalAtDepth': 0, 'uniqueNextDepth': 72}
{'depth': 2, 'uniqueAtDepth': 72, 'cumulativeUniqueToDepth': 82, 'terminalAtDepth': 0, 'uniqueNextDepth': 252}
{'depth': 3, 'uniqueAtDepth': 252, 'cumulativeUniqueToDepth': 334, 'terminalAtDepth': 0, 'uniqueNextDepth': 756}
{'depth': 4, 'uniqueAtDepth': 756, 'cumulativeUniqueToDepth': 1090, 'terminalAtDepth': 0, 'uniqueNextDepth': 8064}
{'depth': 5, 'uniqueAtDepth': 8064, 'cumulativeUniqueToDepth': 9154, 'terminalAtDepth': 120, 'uniqueNextDepth': 61520}
{'depth': 6, 'uniqueAtDepth': 61520, 'cumulativeUniqueToDepth': 69918, 'terminalAtDepth': 1228, 'uniqueNextDepth': 446820}
{'depth': 7, 'uniqueAtDepth': 446820, 'cumulativeUniqueToDepth': 508674, 'terminalAtDepth': 11908, 'uniqueNextDepth': 2276494}
{'depth': 8, 'uniqueAtDepth': 2276494, 'cumulativeUniqueToDepth': 2723648, '

## Is X A Forced Win?

Your observed repeated X wins can happen with deterministic tie-breaking and one fixed depth.

To test whether that is actually a forced win, we run:
- Deterministic self-play across depths.
- Randomized tie-break self-play (choose randomly among equal-best moves).

If O can win in these stronger probes, that is evidence against "X forced win" (though still not a full proof either way).


In [14]:
forced_win_js = r"""
const { createInitialState, getWinner, isDraw, applyAction, getOpponent, getStateKey, getAvailableActions } = require("../../dist/game");
const { getEngineEvaluations } = require("../../dist/minimax");
const { runSelfPlayEpisode } = require("../../dist/learning");
const { DEFAULT_EVALUATION_PLUGIN } = require("../../dist/evaluation");

function deterministicSweep(maxDepth=6){
  const rows=[];
  for(let d=1; d<=maxDepth; d++){
    const ep = runSelfPlayEpisode({depthLimit:d, maxTurns:200, evalPluginX:DEFAULT_EVALUATION_PLUGIN, evalPluginO:DEFAULT_EVALUATION_PLUGIN});
    rows.push({depth:d, winner:ep.winner, turns:ep.turnCount, timeout:!!ep.terminatedByMaxTurns});
  }
  return rows;
}

function chooseWithRandomTiebreak(state, player, history, depthLimit){
  const actions = getAvailableActions(state, player);
  const multiPv = Math.max(1, actions.length);
  const { evaluations } = getEngineEvaluations(state, player, history, depthLimit, multiPv, DEFAULT_EVALUATION_PLUGIN.evaluate);
  if (!evaluations.length) return null;
  const best = evaluations[0].score;
  const tied = evaluations.filter(e => e.score === best);
  return tied[Math.floor(Math.random() * tied.length)].action;
}

function randomTieEpisode(depthLimit, maxTurns=200){
  let state = createInitialState();
  let player = "X";
  const history = new Set([getStateKey(state, player)]);
  for(let turn=0; turn<maxTurns; turn++){
    const w = getWinner(state);
    if (w) return {winner:w, turns:turn};
    if (isDraw(state)) return {winner:null, turns:turn};
    const action = chooseWithRandomTiebreak(state, player, history, depthLimit);
    if (!action) return {winner:null, turns:turn};
    state = applyAction(state, action, player);
    player = getOpponent(player);
    history.add(getStateKey(state, player));
  }
  return {winner:"timeout", turns:maxTurns};
}

function randomTieSweep(depth, episodes){
  const counts={X:0,O:0,draw:0,timeout:0};
  for(let i=0;i<episodes;i++){
    const r = randomTieEpisode(depth, 200);
    if(r.winner==="X") counts.X++;
    else if(r.winner==="O") counts.O++;
    else if(r.winner==="timeout") counts.timeout++;
    else counts.draw++;
  }
  return {depth, episodes, counts};
}

const out = {
  deterministic: deterministicSweep(6),
  random_tie_depth3: randomTieSweep(3, 100),
  random_tie_depth4: randomTieSweep(4, 50)
};

console.log(JSON.stringify(out));
"""

analysis = json.loads(run_node(forced_win_js))
analysis


{'deterministic': [{'depth': 1, 'winner': 'X', 'turns': 7, 'timeout': False},
  {'depth': 2, 'winner': None, 'turns': 200, 'timeout': True},
  {'depth': 3, 'winner': None, 'turns': 200, 'timeout': True},
  {'depth': 4, 'winner': None, 'turns': 200, 'timeout': True},
  {'depth': 5, 'winner': None, 'turns': 200, 'timeout': True},
  {'depth': 6, 'winner': None, 'turns': 200, 'timeout': True}],
 'random_tie_depth3': {'depth': 3,
  'episodes': 100,
  'counts': {'X': 41, 'O': 59, 'draw': 0, 'timeout': 0}},
 'random_tie_depth4': {'depth': 4,
  'episodes': 50,
  'counts': {'X': 15, 'O': 27, 'draw': 0, 'timeout': 8}}}

### Practical conclusion so far

- If you observe O wins at some depth/tie settings, then X is not established as a forced win by current evidence.
- To prove forced win/loss/draw, you would need a full game-theoretic solve (retrograde or complete forward search with exact terminal values).
- The sections above provide a reproducible path to tighten bounds and stress-test the hypothesis incrementally.


## Tablebase Feasibility: Memory And Runtime Estimates

This section estimates feasibility for a tablebase-style solve under different assumptions.

Important caveat:
- A classic tablebase assumes Markov state transitions.
- Your repetition rule depends on full game history, so an exact solution for the current rules is not a plain tablebase.
- The numbers below are best interpreted for a Markov approximation (state = board + active window + side-to-move).


In [15]:
from math import ceil

# State-count scenarios
N_upper = 1_551_615_534  # combinatorial upper bound including side-to-move
N_prefix_lower = 2_723_648  # exact reachable lower bound from depth-limited expansion to ply 8

# Plausible solved-state fractions for scenario analysis
fractions = [1.0, 0.5, 0.25, 0.10]

def bytes_to_human(n: int) -> str:
    units = [(1<<40, "TiB"), (1<<30, "GiB"), (1<<20, "MiB"), (1<<10, "KiB")]
    for u, name in units:
        if n >= u:
            return f"{n/u:.2f} {name}"
    return f"{n} B"

def packed_bytes(states: int, bits_per_state: int) -> int:
    return ceil(states * bits_per_state / 8)

# Concrete encoding options
encodings = [
    ("Outcome only (2 bits)", 2),
    ("Outcome + DTW(8b) = 10 bits", 10),
    ("Outcome + DTW(16b) = 18 bits", 18),
    ("Outcome + best move(16b) = 18 bits", 18),
    ("Outcome + DTW(8b) + best move(16b) = 26 bits", 26),
]

rows = []
for frac in fractions:
    n = int(N_upper * frac)
    for label, bits in encodings:
        b = packed_bytes(n, bits)
        rows.append((frac, n, label, bits, b, bytes_to_human(b)))

for frac, n, label, bits, b, h in rows:
    print(f"fraction={frac:>4.0%} states={n:,} | {label:<42} | {bits:>2} bits/state | {h}")

print()
print("Reference lower bound reached so far:", f"{N_prefix_lower:,}", "states")


fraction=100% states=1,551,615,534 | Outcome only (2 bits)                      |  2 bits/state | 369.93 MiB
fraction=100% states=1,551,615,534 | Outcome + DTW(8b) = 10 bits                | 10 bits/state | 1.81 GiB
fraction=100% states=1,551,615,534 | Outcome + DTW(16b) = 18 bits               | 18 bits/state | 3.25 GiB
fraction=100% states=1,551,615,534 | Outcome + best move(16b) = 18 bits         | 18 bits/state | 3.25 GiB
fraction=100% states=1,551,615,534 | Outcome + DTW(8b) + best move(16b) = 26 bits | 26 bits/state | 4.70 GiB
fraction= 50% states=775,807,767 | Outcome only (2 bits)                      |  2 bits/state | 184.97 MiB
fraction= 50% states=775,807,767 | Outcome + DTW(8b) = 10 bits                | 10 bits/state | 924.83 MiB
fraction= 50% states=775,807,767 | Outcome + DTW(16b) = 18 bits               | 18 bits/state | 1.63 GiB
fraction= 50% states=775,807,767 | Outcome + best move(16b) = 18 bits         | 18 bits/state | 1.63 GiB
fraction= 50% states=775,807,767 | Ou

In [16]:
# Runtime estimates for retrograde-like passes over state records
passes = [1, 3, 5]
throughputs = [100_000, 500_000, 1_000_000, 5_000_000]  # states/sec

def fmt_hours(hours: float) -> str:
    if hours < 1:
        return f"{hours*60:.1f} min"
    if hours < 24:
        return f"{hours:.2f} h"
    return f"{hours/24:.2f} d"

for frac in fractions:
    n = int(N_upper * frac)
    print()
    print(f"State fraction {frac:.0%} ({n:,} states):")
    for p in passes:
        line = []
        for t in throughputs:
            hours = (n * p) / t / 3600
            line.append(f"{t:,}/s -> {fmt_hours(hours)}")
        print(f"  passes={p}: " + " | ".join(line))



State fraction 100% (1,551,615,534 states):
  passes=1: 100,000/s -> 4.31 h | 500,000/s -> 51.7 min | 1,000,000/s -> 25.9 min | 5,000,000/s -> 5.2 min
  passes=3: 100,000/s -> 12.93 h | 500,000/s -> 2.59 h | 1,000,000/s -> 1.29 h | 5,000,000/s -> 15.5 min
  passes=5: 100,000/s -> 21.55 h | 500,000/s -> 4.31 h | 1,000,000/s -> 2.16 h | 5,000,000/s -> 25.9 min

State fraction 50% (775,807,767 states):
  passes=1: 100,000/s -> 2.16 h | 500,000/s -> 25.9 min | 1,000,000/s -> 12.9 min | 5,000,000/s -> 2.6 min
  passes=3: 100,000/s -> 6.47 h | 500,000/s -> 1.29 h | 1,000,000/s -> 38.8 min | 5,000,000/s -> 7.8 min
  passes=5: 100,000/s -> 10.78 h | 500,000/s -> 2.16 h | 1,000,000/s -> 1.08 h | 5,000,000/s -> 12.9 min

State fraction 25% (387,903,883 states):
  passes=1: 100,000/s -> 1.08 h | 500,000/s -> 12.9 min | 1,000,000/s -> 6.5 min | 5,000,000/s -> 1.3 min
  passes=3: 100,000/s -> 3.23 h | 500,000/s -> 38.8 min | 1,000,000/s -> 19.4 min | 5,000,000/s -> 3.9 min
  passes=5: 100,000/s ->

In [17]:
# If you store explicit transitions (often unnecessary for retrograde), memory can dominate.
# Estimate with average branching factor and 4-byte destination IDs.
avg_branching = [8, 16, 24, 32]
dest_bytes = 4

for frac in fractions:
    n = int(N_upper * frac)
    print()
    print(f"Transition storage at fraction {frac:.0%} ({n:,} states):")
    for b in avg_branching:
        bytes_needed = n * b * dest_bytes
        print(f"  avg branching={b:>2}: {bytes_to_human(bytes_needed)}")



Transition storage at fraction 100% (1,551,615,534 states):
  avg branching= 8: 46.24 GiB
  avg branching=16: 92.48 GiB
  avg branching=24: 138.73 GiB
  avg branching=32: 184.97 GiB

Transition storage at fraction 50% (775,807,767 states):
  avg branching= 8: 23.12 GiB
  avg branching=16: 46.24 GiB
  avg branching=24: 69.36 GiB
  avg branching=32: 92.48 GiB

Transition storage at fraction 25% (387,903,883 states):
  avg branching= 8: 11.56 GiB
  avg branching=16: 23.12 GiB
  avg branching=24: 34.68 GiB
  avg branching=32: 46.24 GiB

Transition storage at fraction 10% (155,161,553 states):
  avg branching= 8: 4.62 GiB
  avg branching=16: 9.25 GiB
  avg branching=24: 13.87 GiB
  avg branching=32: 18.50 GiB


### Readout

- Memory for *value-only* storage is modest even near the upper bound (2 bits/state packed).
- Runtime is likely the harder limit unless you parallelize and avoid storing full transition graphs.
- For exact current rules (history-based repetition), expect a much harder problem than these numbers suggest.
- A practical path is solving the Markov approximation first, then evaluating how often repetition-history changes move legality.
